# California Clear Sterile Compound License Enrichment

## Notebook Config

In [60]:
import pandas as pd
import re 
import numpy as np



# Set pandas to display more rows/columns for wider dataframes
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)     # Adjust rows
pd.set_option('display.width', 1000)       # Increase width for better visibility

In [61]:
# Load Data
pharmacies = pd.read_csv("all_pharmacies.csv", dtype=str)
# Convert ExpiratioN Date to Date
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'])

In [62]:
zips = pd.read_csv("zip_lat_long.csv", dtype=str)

## Enrich Location Data

In [63]:
def get_lat_lon(df_to_edit: pd.DataFrame, df_with_zips: pd.DataFrame) -> pd.DataFrame:
    merged_df = df_to_edit.merge(df_with_zips, on='Zip', how="left")
    return merged_df

In [64]:
def update_state_to_abbreviation(df):

    state_name_to_abbrev = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
        'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
        'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
        'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
        'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
        'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
        'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
        'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    }
    df['State'] = df['State'].map(state_name_to_abbrev)
    return df


In [65]:
def replace_out_of_state_with_nan(df):
    df['County'] = df['County'].replace('OUT OF STATE', np.nan)
    return df

In [66]:
pharmacies = get_lat_lon(pharmacies, zips)
pharmacies = update_state_to_abbreviation(pharmacies)
pharmacies = replace_out_of_state_with_nan(pharmacies)
pharmacies.head()

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG
0,986 INFUSION PHARMACY #1 INC,LSC 101414,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ANAHEIM,CA,ORANGE,92807,33.848733,-117.788357
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,2025-03-01,SALINAS,CA,MONTEREY,93901,36.636344,-121.623978
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,WARRENDALE,PA,NaN,15086,40.674947,-80.106364
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,ORLANDO,FL,NaN,32822,28.489898,-81.290154
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ESCONDIDO,CA,SAN DIEGO,92025,33.102005,-117.080419


## 

## Identify & Remove Duplicate Licenses

In [67]:
def remove_previous_names(df):

    # Remove rows where 'Pharmacy Name' contains '(Previous Name)'
    df = df[~df['Pharmacy Name'].str.contains(r'\(Previous Name\)', na=False)]
    
    return df

In [68]:
pharmacies = remove_previous_names(pharmacies)

In [69]:
# Show rows with duplicate License Numbers
duplicates = pharmacies[pharmacies.duplicated(subset='License Number', keep=False)]
duplicates


,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG


In [70]:
previous_name_rows = pharmacies[pharmacies['Pharmacy Name'].str.contains(r'\(Previous Name\)', na=False)]
previous_name_rows

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG


## Enrich Pharmacies with Category & Typing Data

In [89]:
def identify_hospitals(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying hospitals and medical facilities
    hospital_keywords = [
        'ADVENTIST HEALTH',
        'BOWYER PHARMACY',
        'CAMPUS',
        'CANCER CENTER',
        'CARDINAL HEALTH',
        'CARE PAVILION',
        'CEDARS-SANAI',
        'CEDARS-SINAI',
        'CENTER FOR',
        'CITY OF HOPE',
        'CLINIC',
        'CONVERSIO HEALTH',
        'CORRECTIONAL FACILITY',
        'COUNTY ONCOLOGY',
        'CRMC',
        'EMANATE HEALTH',
        'HEALTH CENTER',
        'HEALTH SERVICES',
        'HEALTH SYSTEM',
        'HEALTHCARE DISTRICT',
        'HIGHLAND CARE PAVILION',
        'HOAG',
        'HOSPITAL',
        'HOSPTIAL',
        'HSP',
        'KAISER',
        'KAWEAH HEALTH',
        'LPCH',
        'LUNDQUIST INSTITUTE',
        'MED CTR',
        'MEDICAL CENTER',
        'MEMORIAL',
        'MOUNTAIN VIEW PHARMACEUTICALS',
        'OF USC',
        'ONCOLOGY CENTER',
        'OUTPATIENT',
        'PENTEC HEALTH',
        'PRIME HEALTHCARE',
        'PROVIDENCE ONCOLOGY',
        'RADY CHILDREN\'S',
        'REGIONAL',
        'REHABILITATION',
        'RIVER\'S EDGE',
        'RUHS',
        'SCRIPPS',
        'SHRINERS',
        'SOLEO HEALTH INC',
        'SONOMA SPECIALTY PHARMACY',
        'STANFORD HEALTH CARE',
        'SUTTER LAKESIDE',
        'TODD CANCER PAVILION',
        'UC DAVIS',
        'UCI HEALTH',
        'CHILDRENS VILLAGE PHARMACY',
        'CONSONUS PHARMACY SERVICES CA NORTH LLC',
        'CRESCENT HEALTHCARE INC DBA CRESCENT HEALTHCARE',
        'GMT PHARMACY',
        'HIGHLAND CARE PAVILLION PHARMACY'
    ]

    # Create a regex pattern to match any of the hospital keywords
    keywords_pattern = '|'.join([re.escape(keyword) for keyword in hospital_keywords])

    # Use .loc[] to safely assign "Medical Facility" where conditions are met
    df.loc[df['Pharmacy Name'].str.contains(keywords_pattern, case=False, regex=True, na=False), 'Facility Type'] = 'Medical Facility'

    return df


def identify_veterinary_only(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying veterinary-only pharmacies
    vet_keywords = [
        'PETNET PHARMACEUTICAL',
        'PROLAB PHARMACY',
        'SPECIALTY VETERINARY PHARMACY',
        'WEDGEWOOD PHARMACY LLC',
        'BCP VETERINARY PHARMACY',
        'COVETRUS NORTH AMERICA, LLC']

    # Use .loc[] to safely assign "Veterinary Only" where conditions are met
    df.loc[df['Pharmacy Name'].apply(lambda x: any(keyword in x for keyword in vet_keywords)), 'Facility Type'] = 'Veterinary Only'

    return df


def identify_infusion_centers(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying infusion centers
    infusion_center_search_terms = [
        'INFUSION',
        'HOSPICARE',
        'INTEGRATED CARE SYSTEMS',
        'IV LEAGUE',
        'KABAFUSION',
        'MARSHALL MEDICAL ONCOLOGY PHARMACY',
        'MODEL DRUG',
        'NUTRISHARE',
        'OMNICARE',
        'OPTION CARE',
        'OPTUM',
        'OSO HOME CARE',
        'OWL REXALL DRUG',
        'PLAZA HOME CARE',
        'POLARIS PHARMACY SERVICES',
        'PREFERRED EXCELLENT CARE',
        'QUADRACARE',
        'SCRIPTS',
        'SHARP CENTRAL PHARMACY SERVICES',
        'TRINITY CARE RX',
        'VIVE RX',
        'ACCREDO HEALTH GROUP INC',
        'ALERACARE PHARMACY CALIFORNIA',
        'AMERIPHARMA',
        'BIOMED CALIFORNIA INC',
        'BROOKS HEALTH CARE',
        'CALIFORNIA SPECIALTY PHARMACY LLC',
        'ELEVARE SPECIALTY SERVICES',
        'FIRST CALL PHARMACY LLC',
        'CVS/SPECIALTY OR CARELONRX SPECIALTY PHARMACY',
        'CORAM HEALTHCARE CORPORATION OF NEVADA DBA CORAM CVS/SPECIALTY IN',
        'CAREMARK ILLINOIS SPECIALTY PHARMACY, LLC DBA CVS/SPECIALTY OR',
        'ALLIANCERX WALGREENS PHARMACY',
        'PHARMERICA'
    ]
   
    # Create a regex pattern to match any of the infusion center keywords
    pattern = '|'.join(infusion_center_search_terms)

    # Use .loc[] to safely assign "Infusion Center" based on conditions in License Type or Pharmacy Name
    df.loc[df['License Type'].str.contains('Infusion', case=False, na=False) | 
           df['Pharmacy Name'].str.contains(pattern, case=False, na=False), 'Facility Type'] = 'Infusion Center'

    return df

def identify_research_centers(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying veterinary-only pharmacies
    research_keywords = [
        'UCSD RESEARCH PHARMACY'
    ]

    # Use .loc[] to safely assign "Veterinary Only" where conditions are met
    df.loc[df['Pharmacy Name'].apply(lambda x: any(keyword in x for keyword in research_keywords)), 'Facility Type'] = 'Research Center'

    return df

def identify_pharmacies(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying veterinary-only pharmacies
    verified_pharmacies = {
        'A & O SPECIALTY PHARMACY' : {
            'specialties' : ['Hormone Replacement Therapy (HRT)', 'Veterinary', 'Dermatology', 'Pain Management', 'Gastroenterology', 'Ophthalmology', 'Oncology'],
            'conditions' : ['Erectile Dysfunction', 'Thyroid Dysfunction'],
            'registered outsourcer' : False,
            'accreditations' : ['PCCA', 'IACP', 'PCAB'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.aocompounding.com'
        },
        'ADVANCE CARE PHARMACY' : {
            'specialties' : ['Hospice', 'Long Term Care', 'Pain Management', 'Antibiotics', 'Total Parenteral Nutrition (TPN)'],
            'conditions' : ['Terminally Ill',],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.advancecarepharmacy.com'
        },
        'AMBROSIA HEALTHCARE INC' : {
            'specialties' : ['Veterinary', 'Pain Management', 'Hospice', 'Hormone Replacement Therapy (HRT)', 'Long Term Care'],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : ['IACP'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.ambrosiahc.com'
        },
        'BOOTHWYN PHARMACY LLC' : {
            'specialties' : ['Veterinary', 'Ophthalmology', 'Weight Loss', 'Hormone Replacement Therapy (HRT)', 'Dermatology', 'Pediatrics', 'Pain Management' 'Gastroenterology', 'Podiatry'],
            'conditions' : ['Infertility', 'Glaucoma', 'Dry Eye Syndrome', 'Acne', 'Psoriasis', 'Eczema'],
            'registered outsourcer' : False,
            'accreditations' : ['PCAB', 'ACHC', 'LegitScript'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://boothwyn.com'
        },
        'CD PHARMACY LLC DBA RED ROCK PHARMACY' : {
            'specialties' : ['Hospice', 'Long Term Care', 'Weight Loss'],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.redrockrx.com'
        },
        'CENTRAL ADMIXTURE PHARMACY SERVICES INC' : {
            'specialties' : ['Total Parenteral Nutrition (TPN)', 'Anesthesia'],
            'conditions' : ['Cardioplegia'],
            'registered outsourcer' : True,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.capspharmacy.com/en.html'
        },
        'CHEMIQUE PHARMACEUTICALS INC' : {
            'specialties' : ['IV Vitamin Therapy', 'Total Parenteral Nutrition (TPN)', 'Weight Loss', 'Antibiotics', 'Hormone Replacement Therapy (HRT)', 'Veterinary', 'Oncology', 'Pain Management'],
            'conditions' : ['Anemia', 'Nausea', 'Diabetes', 'Hypertension'],
            'registered outsourcer' : False,
            'accreditations' : ['American Association of Homeopathic Pharmacists', 'ACHC', 'ASC' , 'APhA', 'ASHP', 'ASPEN', 'NHIA', 'PDA'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://chemiquepharm.com'
        },
        'CITRUS PHARMACY' : {
            'specialties' : ['Long Term Care'],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://citrusrx.com'
        },
        'CLINISHARE PHARMACY' : {
            'specialties' : [''],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.rxless.com/pharmacies/california/westhills/clinishare-pharmacy/23719-Roscoe-Blvd/1477638765'
        },
        'COAST QUALITY PHARMACY LLC D/B/A ANAZAOHEALTH' : {
            'specialties' : ['IV Vitamin Therapy', 'Pain Management', 'Weight Management', 'Hormone Replacement Therapy (HRT)', 'Hair Care', 'Nuclear Medicine', 'Anti-Aging', 'Sexual Health', 'Aesthetics'],
            'conditions' : [''],
            'registered outsourcer' : True,
            'accreditations' : ['LegitScript'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.anazaohealth.com'
        },
        'CURE STAT RX' : {
            'specialties' : ['IV Vitamin Therapy', 'Oncology', 'Hematology', 'Cardiology', 'Immunotherapy', 'Total Parenteral Nutrition (TPN)', 'Chronic Disease', 'Opthalmology', 'Veterinary'],
            'conditions' : ['Electrolyte Imbalance', 'Appetite', 'Infertility'],
            'registered outsourcer' : False,
            'accreditations' : ['URAC', 'ACHC', 'NHIA', 'PCAB'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://curestatrx.com'
        },
        'DEL\'S PHARMACY I' : {
            'specialties' : ['Long Term Care', 'Hospice', 'Total Parenteral Nutrition (TPN)'],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://delspharmacy.com'
        },
        'DIAMOND DRUGS INC' : {
            'specialties' : ['Long Term Care', 'Correctional Populations', ],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.diamondpharmacy.com'
        },
        'DWAYNE\'S FRIENDLY PHARMACY' : {
            'specialties' : [''],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://dwaynespharmacy.com'
        },
        'DYNAMIC RX LABS LLC' : {
            'specialties' : ['IV Vitamin Therapy', 'Weight Loss', 'Hormone Replacement Therapy (HRT)', 'Hair Care', 'Sexual Health'],
            'conditions' : ['Erectile Dysfunction', 'Premature Ejaculation'],
            'registered outsourcer' : False,
            'accreditations' : ['LegitScript'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://dynamicrxlabs.com'
        },
        'ENCINO PHARMACY' : {
            'specialties' : [''],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://www.encinopharmacy.com'
        },
        'ENOVEX PHARMACY' : {
            'specialties' : ['Pain Management', 'Hormone Replacement Therapy (HRT)', 'Veterinary', 'Dermatology'],
            'conditions' : ['Arthritis'],
            'registered outsourcer' : False,
            'accreditations' : ['PCAB', 'PCCA', 'A4PC'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://enovexrx.com'
        },
        'FIRESIDE PHARMACY' : {
            'specialties' : ['IV Vitamin Therapy', 'Hormone Replacement Therapy (HRT)', 'Sexual Health', 'Weight Loss', 'Anti-Aging', 'Dentistry', 'Dermatology', 'Ear, Nose, and Throat', 'Pain Management', 'Palliative Care', 'Pediatrics', 'Podiatry', 'Sports Medicine', 'Veterinary', 'Wound Care'],
            'conditions' : ['Erectile Dysfunction'],
            'registered outsourcer' : False,
            'accreditations' : ['PCCA'],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : 'https://firesidepharmacy.com'
        },
        'Next Pharmacy' : { # TODO:  Pickup at Fireside checking each website and completing this 
            'specialties' : [''],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : ''
        },
        'Next Pharmacy' : {
            'specialties' : [''],
            'conditions' : [''],
            'registered outsourcer' : False,
            'accreditations' : [''],
            'facility type' : 'Sterile Compounding Pharmacy',
            'url' : ''
        }
        
    }
    

    # TODO:  Update the pharmacy table to accmodate all of these verified pharmacies and their info.  

    return df

In [92]:
def categorize_pharmacies(df) -> pd.DataFrame:

    # Create an isGovernment column and populate it based on the License Type
    df['isGovernment'] = df['License Type'].str.contains('Government', case=False, na=False)
    
    # Create an isSatellite column and populate it based on the License Type
    df['isSatellite'] = df['License Type'].str.contains('Satellite', case=False, na=False)
    
    df = identify_hospitals(df)
    df = identify_infusion_centers(df)
    df = identify_veterinary_only(df)
    df = identify_research_centers(df)
    return df

In [93]:
pharmacies = categorize_pharmacies(pharmacies)

# Filter rows where 'Facility Type' is either NaN or an empty string
empty_facility_type = pharmacies[pharmacies['Facility Type'].isna() | (pharmacies['Facility Type'] == '')]

# Display the result
empty_facility_type


,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG,isGovernment,isSatellite,Facility Type
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,2025-03-01,SALINAS,CA,MONTEREY,93901,36.636344,-121.623978,False,False,NaN
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ESCONDIDO,CA,SAN DIEGO,92025,33.102005,-117.080419,False,False,NaN
37,AMBROSIA HEALTHCARE INC,LSC 99935,Sterile Compounding Pharmacy,CLEAR,2025-06-01,PALM DESERT,CA,RIVERSIDE,92211,33.765401,-116.334205,False,False,NaN
61,BOOTHWYN PHARMACY LLC,NSC 101082,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-07-01,KENNETT SQUARE,PA,NaN,19348,39.870280,-75.712913,False,False,NaN
83,CD PHARMACY LLC DBA RED ROCK PHARMACY,NSC 101839,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-03-01,ST. GEORGE,UT,NaN,84790,37.044186,-113.577299,False,False,NaN
95,CENTRAL ADMIXTURE PHARMACY SERVICES INC,LSC 99040,Sterile Compounding Pharmacy,CLEAR,2025-09-01,FREMONT,CA,ALAMEDA,94538,37.501368,-121.956753,False,False,NaN
96,CENTRAL ADMIXTURE PHARMACY SERVICES INC,LSC 101206,Sterile Compounding Pharmacy,CLEAR,2025-10-01,SANTA FE SPRINGS,CA,LOS ANGELES,90670,33.933565,-118.062611,False,False,NaN
100,CHEMIQUE PHARMACEUTICALS INC,LSC 99028,Sterile Compounding Pharmacy,CLEAR,2025-12-01,POMONA,CA,LOS ANGELES,91768,34.063516,-117.790379,False,False,NaN
101,CHEMIQUE PHARMACEUTICALS INC,LSC 99027,Sterile Compounding Pharmacy,CLEAR,2025-06-01,WHITTIER,CA,LOS ANGELES,90602,33.972782,-118.022066,False,False,NaN
120,CITRUS PHARMACY,LSC 99560,Sterile Compounding Pharmacy,CLEAR,2025-07-01,REDLANDS,CA,SAN BERNARDINO,92373,34.011953,-117.159699,False,False,NaN
